In [1]:
# general house keeping
import warnings
warnings.filterwarnings('ignore')

In [2]:
# audio to text pipeline
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC
import librosa as lb
import torch

In [3]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained('facebook/wav2vec2-base-960h')
model = Wav2Vec2ForCTC.from_pretrained('facebook/wav2vec2-base-960h')

# Save them locally
tokenizer.save_pretrained('models/tokenizer-speechbrain')
model.save_pretrained('models/model-speechbrain')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2

In [2]:


def audiototext(wav_file_path, sampling_rate = 16000):
    waveform, rate = lb.load(wav_file_path, sr = sampling_rate)
    input_values = tokenizer(waveform, return_tensors='pt').input_values
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)
    
    return str(transcription[0]).lower()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2

In [13]:
# text to audio pipeleine
import torchaudio
from speechbrain.inference.TTS import Tacotron2
from speechbrain.inference.vocoders import HIFIGAN
import os

# Intialize TTS (tacotron2) and Vocoder (HiFIGAN)
tacotron2 = Tacotron2.from_hparams(source="tmpdir_tts")
hifi_gan = HIFIGAN.from_hparams(source="tmpdir_vocoder")


def texttoaudio(text, out_filename):
    mel_output, mel_length, alignment = tacotron2.encode_text(text)
    waveforms = hifi_gan.decode_batch(mel_output)
    
    output_dir = "outputs"
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # os.path.join(output_dir, "test.wav")
    file_name = "outputs" + "/" + out_filename + ".wav"
    torchaudio.save(file_name, waveforms.squeeze(1), 22050)
    print(f"File save to: {file_name}")

In [4]:
sample_audio = "data/outputs/result-speechbrain.wav" # Should be wav (waveform) file
sample_outpath = "test"

In [5]:
transcribed_text = audiototext(sample_audio)
print(transcribed_text)

2024-06-23 12:19:47.165378: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


welcome to jackoranda health's solutions


In [9]:
print(type(transcribed_text))
print(type(sample_outpath))

<class 'str'>
<class 'str'>


In [14]:
texttoaudio(transcribed_text, sample_outpath)

File save to: outputs/test.wav
